# Chapter 4 Djangoを用いたWebサイトの作成

## 1. Djangoの環境設定
まずはDjangoの環境を設定します。以下のコマンドでDjangoをインストールします。
Django は Python で Web アプリケーションを作成する場合に一番メジャーなフレームワークです。

In [1]:
# Django のインストール
!pip install django

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 27.9 MB/s eta 0:00:00


## 2. Djangoプロジェクトの作成
次にDjangoのプロジェクトを作成します。以下のコマンドでプロジェクトを作成します。

In [2]:
# Django のプロジェクト作成
!django-admin startproject mysite

## 3. Djangoアプリケーションの作成
プロジェクト内にアプリケーションを作成します。以下のコマンドでアプリケーションを作成します。

In [3]:
# myapp アプリケーションの作成
!cd mysite && python manage.py startapp myapp

## 3.1 Google Colab 上で Django を動かすための設定
ALLOWD_HOSTS に colab.research.google.com を加える。この設定により google colab 上で Django アプリケーションが動作可能になる

In [4]:
# Django を GoogleColab 上で動作させるための準備
# Django のネットワークを設定する -> colab.research.google.com に設定する
# ALLOWED_HOSTS = ['colab.research.google.com']
!cd mysite/mysite && sed -i.bak -e 's/\(ALLOWED_HOSTS\).*/\1 = ["colab\.research\.google\.com"]/g' settings.py
!cd mysite/mysite && grep ALLOWED_HOSTS settings.py

ALLOWED_HOSTS = ["colab.research.google.com"]


## 3.2 Google Colab 上で動作する Django アプリケーションを外部から見えるためにプロキシ設定



In [5]:
# プロキシの設定
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://9no5k3kzojt-496ff2e9c6d22116-8000-colab.googleusercontent.com/


## 3.3 Django アプリケーションを試す

In [ ]:
# Django サーバーの起動
!cd mysite && python manage.py runserver 8000
# サイトが確認出来たら停止ボタンを押す。

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
January 23, 2024 - 13:01:55
Django version 5.0.1, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.

[23/Jan/2024 13:01:55] "GET / HTTP/1.1" 200 10629
Not Found: /favicon.ico
[23/Jan/2024 13:01:56] "GET /favicon.ico HTTP/1.1" 404 2121


## 4. データベースの設定
今回は SQLite を使用するのでデフォルトのままにする

settings.py のデータベースの設定項目は以下。
現状は SQLite3 を使用する方法になっている。
```
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}
```
例えば PostgreSQL を使用する場合は以下のような感じになる。
※ 下記は仮の値になっているため、必要な値は適当に追加する。
```
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'NAME': 'dbname',
        'USER': 'username',
        'PASSWORD': 'password',
        'HOST': 'localhost',
        'PORT': '',
    }
}
```





## 5. ビューの作成
DjangoでWebページを表示するためのビューを作成します。views.pyを編集します。

views は表示のための前処理をする


主にデータベースからデータの取得、整理などを行う。

ユーザからリクエスト (request) を受け取り、データの取得を行い Django のテンプレートシステムへ渡す。

In [6]:
# 変数等設定。
import os
from pathlib import Path

basedir = Path(os.getcwd())
appdir = basedir / "mysite" / "myapp"
configdir = basedir / "mysite" / "mysite"

# views.py
code = """
from django.shortcuts import render
from .models import WeatherData

def index(request):
    data = WeatherData.objects.all()
    return render(request, 'index.html', {'data': data})

"""

with open(appdir / "views.py", "w") as f:
  f.write(code)

## 5.1 モデルの作成
データベースの定義ファイルを作成する

models.py はデータベース定義のファイル。
Django は ORM という機能があり、class を書くようにデータベースの定義をすることができる。

In [7]:
# models.py
code = """
from django.db import models

class WeatherData(models.Model):
    date = models.DateField()
    temperature = models.FloatField()
    rainfall = models.FloatField()
"""
with open(appdir / "models.py", "w") as f:
  f.write(code)

## 6. URLのルーティング
URLをビューに接続します。urls.pyを編集します。

urls.py はルーティングを定義するファイル。

ユーザがアクセスした URL に対して、対応したビューを返す役目。

In [13]:
# urls.py
code = """
from django.urls import path
from myapp import views

urlpatterns = [
    path('', views.index, name='index'),
]
"""
with open(configdir / "urls.py", "w") as f:
  f.write(code)


## 7. テンプレートの作成
Webページの見た目を決めるテンプレートを作成します。templatesディレクトリにhtmlファイルを作成します。

テンプレートシステムは views から受け取ったデータを html としてユーザに返却する仕組み

In [24]:
# template index.html
(appdir / "templates").mkdir(exist_ok=True)
(appdir / "templates" / "myapp").mkdir(exist_ok=True)

html = """
<html>
 <head>
   <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
 </head>
 <body>
 <div>
  <h1>Chapter 4 Djangoを用いたWebサイトの作成</h1>
 </div>
  <div>
   <table class="table">
    <th>
     <tr>
      <td>date</td>
      <td>temperature</td>
      <td>rainfall</td>
     </tr>
    </th>
    <tb>
     {% for i in data %}
     <tr>
      <td>{{ i.date }}</td>
      <td>{{ i.temperature }}</td>
      <td>{{ i.rainfall }}</td>
     </tr>
     {% endfor %}
    </tb>
  </table>
 </body>
</html>
"""

with open(appdir / "templates" / "index.html", "w") as f:
  f.write(html)

## 7.2 Django アプリケーションに作成した myapp を追加する
Django は明示的に作成したアプリケーションを設定に反映する必要がある。

明示的にアプリケーションを追加する仕組みにより、Django のアプリケーションはモジュール化が可能になります。
知らない人が作ったアプリケーションを追加したりなど、逆に自分が作ったアプリケーションを他人に共有することが容易になっている。

参考: https://djangopackages.org/

In [10]:
# settings.py に myapp を追加する
!cd mysite/mysite && sed -i.bak "/INSTALLED_APPS = \[/a \    'myapp\.apps\.MyappConfig'," settings.py

## 7.3 データベースにモデルを反映させる

Django は models.py を書いただけではデータが入らない。明示的にアプリケーションを指定してデータベースを書き込む。

makemigrations で models.py から中間ファイルを出力し
migrate で models.py の内容を反映させる。

In [11]:
!cd mysite && python manage.py makemigrations myapp
!cd mysite && python manage.py migrate

Migrations for 'myapp':
  myapp/migrations/0001_initial.py
    - Create model WeatherData
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, myapp, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  A

## 8. Djangoサーバーの起動
最後にDjangoのサーバーを起動して、Webサイトが正しく動作するか確認します。

In [25]:
# Django サーバーの起動
!cd mysite && python manage.py runserver 8000

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
January 26, 2024 - 08:00:37
Django version 5.0.1, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.

[26/Jan/2024 08:00:38] "GET / HTTP/1.1" 200 616
Not Found: /favicon.ico
[26/Jan/2024 08:00:38] "GET /favicon.ico HTTP/1.1" 404 2129


In [18]:
import sys
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


sys.path.append(str(basedir / "mysite"))
os.environ.setdefault("DJANGO_SETTINGS_MODULE","mysite.settings")

django.setup()

from myapp.models import WeatherData

data = WeatherData(date="2023-01-01", temperature=25.5, rainfall=100)
data.save()